In [1]:
import os
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset,random_split
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchvision.datasets import ImageFolder
from tqdm.notebook import trange, tqdm
from torchsummary import summary

In [2]:
device = torch.device('cuda')
torch.manual_seed(42)

In [3]:
class LoadData(Dataset):
    def __init__(self,root_dir,transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.data = []
        self.labels =[]
        self.classes = sorted(os.listdir(root_dir))
        self.classes_to_idx = {cls:idx for idx,cls in enumerate(self.classes)}

        for cls in self.classes:
            cls_path = os.path.join(root_dir,cls)
            for img in os.listdir(cls_path):
                img_path = os.path.join(cls_path,img)
                self.data.append(img_path)
                self.labels.append(self.classes_to_idx[cls])
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,idx):
        img_path = self.data[idx]
        label = self.labels[idx]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

        if self.transform:
            img = self.transform(img)
        return img,label

In [4]:
directory = "/kaggle/input/four-shapes/shapes"
transform = transforms.Compose([
    transforms.ToPILImage(), 
    transforms.ToTensor(),
])
dataset = LoadData("/kaggle/input/four-shapes/shapes",transform=transform)

In [5]:
train_size = int(0.8 * len(dataset))
val_size = int(len(dataset) - train_size)

train_ds,val_ds = random_split(dataset,[train_size,val_size])

In [6]:
train_ds[0][0].shape

torch.Size([3, 200, 200])

In [7]:
batch_size=128

train_loader = DataLoader(train_ds,batch_size=128,shuffle=True)
val_loader = DataLoader(val_ds,batch_size=128)

In [8]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(32*100*100,128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128,4)

    def forward(self,x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [9]:
cnn = CNN()
cnn = cnn.to(device)
summary(cnn,input_size=(3,200,200))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 200, 200]             896
              ReLU-2         [-1, 32, 200, 200]               0
         MaxPool2d-3         [-1, 32, 100, 100]               0
           Flatten-4               [-1, 320000]               0
            Linear-5                  [-1, 128]      40,960,128
              ReLU-6                  [-1, 128]               0
            Linear-7                    [-1, 4]             516
Total params: 40,961,540
Trainable params: 40,961,540
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.46
Forward/backward pass size (MB): 24.42
Params size (MB): 156.26
Estimated Total Size (MB): 181.13
----------------------------------------------------------------


In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn.parameters(),lr=0.001)
epochs = 8

for epoch in range(epochs):  
    cnn.train()
    t_loss = 0
    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)
        
        outputs = cnn(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        torch.cuda.synchronize()
        optimizer.step()
        t_loss += loss.item()
    print(f"Epoch : {epoch+1}, Loss : {t_loss}")

  0%|          | 0/94 [00:00<?, ?it/s]

Epoch : 1, Loss : 188.51261753588915


  0%|          | 0/94 [00:00<?, ?it/s]

Epoch : 2, Loss : 2.5727155837230384


  0%|          | 0/94 [00:00<?, ?it/s]

...

Epoch : 6, Loss : 0.26837182900635526


  0%|          | 0/94 [00:00<?, ?it/s]

Epoch : 7, Loss : 0.19122024829266593


  0%|          | 0/94 [00:00<?, ?it/s]

Epoch : 8, Loss : 0.1499667783791665


In [11]:
cnn.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs,labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)

    outputs = cnn(inputs)
    _, preds = torch.max(outputs,1)

In [12]:
def accuracy(preds,labels):
    acc = (preds==labels).sum().item() / labels.size(0)
    return acc
print(f"Accuracy: {accuracy(preds,labels)}")
print(f"Loss: {loss}")

Accuracy: 1.0
Loss: 0.0004930994473397732
